In [1]:
import igl

In [2]:
#https://github.com/libigl/libigl-tutorial-data

In [3]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time

In [4]:
V, F = igl.read_triangle_mesh('data/arm.obj')
C,BE,_,_,_,_ = igl.read_tgf('data/arm.tgf')
W = igl.read_dmat('data/arm-weights.dmat')
# labels = np.load('data/hand.label.npy').astype(int)
# v -= v.min(axis=0)
# v /= v.max()

In [5]:
TV = np.concatenate((V, C))
TE = BE + V.shape[0]

In [6]:
labels = np.concatenate((np.zeros(V.shape[0]), np.array(range(1,C.shape[0]+1)))).astype(int)

In [9]:
handle_vertex_positions = TV.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = TV[slices] + np.array([[x,y,z]])
    # v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    # p.update_object(vertices = v_deformed)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [10]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [11]:
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    return target_pos
''' (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer
'''

' (Optional) Register this function to perform interactive deformation\npos_f.deformer = position_deformer\n'

In [12]:

## Widget UI

p = mp.plot(TV, F)
# p = mp.plot(v,f)
# p.add_points(VE)
p.add_edges(C,BE, shading={"line_color": "green"});
iw.interact(pos_f,
            **widgets_wrapper())

/Users/davidbelgrod/miniconda3/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0001610…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4, 5), value=1), FloatSlider(value=0.0, desc…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

In [13]:
handle_vertex_positions

array([[-0.692193  ,  0.084945  ,  0.0399108 ],
       [-0.693609  ,  0.090783  ,  0.0441324 ],
       [-0.707763  ,  0.0805516 ,  0.038791  ],
       ...,
       [ 0.08140069,  0.18519857, -0.04254464],
       [ 0.5987815 , -0.07760989,  0.06525213],
       [ 0.73572894, -0.17523362,  0.09021015]])

In [14]:
C.shape

(5, 3)

In [ ]:
W.shape

In [ ]:
# W is expressed in terms of bones
# So we need to determine rotation of each bone then multiply by weights and sum to get new vertices
# As we move the vertices up down, rotate we have to recalculate the bone rotations and then above

In [ ]:
# // vQ is a list of rotations as quaternions
# // vT is a list of translations
# igl::dqs(V,W,vQ,vT,U);

In [75]:
# VE,EE,P,BE,CE,PE
Q = igl.directed_edge_orientations(C,BE);
# Q #E list of quaternions

In [76]:
Q.shape

(4, 4)

In [73]:
W.shape

(8311, 4)

In [34]:
Q.shape
Vnorm = np.linalg.norm(Vprime, axis = 1).reshape((-1,1))

In [36]:
Vprime = (W @ Q) / np.repeat(Vnorm, BE.shape[0], axis = 1)

(8311, 4)

In [50]:
import quaternion
QQ = quaternion.as_quat_array(Q)

In [56]:
V.shape

(8311, 3)

In [57]:
W.shape


(8311, 4)

In [68]:
V

array([[-0.692193 ,  0.084945 ,  0.0399108],
       [-0.693609 ,  0.090783 ,  0.0441324],
       [-0.707763 ,  0.0805516,  0.038791 ],
       ...,
       [-0.891664 ,  0.0875205, -0.0301159],
       [-0.814435 ,  0.161851 , -0.152605 ],
       [-0.831258 ,  0.123607 , -0.141994 ]])

In [69]:
V.shape

(8311, 3)

In [70]:
Q.shape

(4, 4)

In [153]:
TQ = np.repeat([[1,0,0,0]], 4, axis = 0)
TQ = np.array(TQ)
TQ

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]])

In [154]:
Vprime = V.copy()
for i in range(V.shape[0]):
    vi = V[i,:]
    wi = W[i,:]
    qi = np.diag(wi) @ TQ
    
    wqi  = np.sum(qi, axis=0)
    wqi /= np.linalg.norm(ri)
    
    vi_q = quaternion.as_quat_array(np.concatenate(([0], vi)))
    
    qi_q = quaternion.as_quat_array(wqi)
    
    # print(ri)
    
    res = qi_q * vi_q * qi_q.inverse()
    
    Vprime[i] = [res.x, res.y, res.z]
    
    # print(ri.shape)
    
    

In [155]:
p = mp.plot(Vprime, F)
p.add_mesh(V+.5, F)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0001610…

1

In [82]:
x = np.array([[2,2], 
              [4,4]])

In [85]:
np.sum(x,axis=1)

array([4, 8])

In [117]:
y = np.array([0,1,1,2])

In [96]:
np.concatenate(([0], [1,2]))

array([0, 1, 2])

In [114]:
qq  = quaternion.as_quat_array(Q[0,:])

In [123]:
res = qq *  quaternion.as_quat_array(y) * qq.inverse()

In [125]:
res

quaternion(1.11022302462516e-16, -2.2286297134392, 1.01033982841775, -0.111458653718282)

In [126]:
l = np.array((res.x, res.y, res.z))

In [127]:
l

array([-2.22862971,  1.01033983, -0.11145865])